In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random
import string 
import re
from scipy import spatial
from sklearn.manifold import TSNE
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk import pos_tag
import os
from sklearn.naive_bayes import GaussianNB
from sklearn.feature_extraction.text import TfidfVectorizer

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
from nltk.corpus import wordnet
from nltk.corpus import stopwords
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
stopword = nltk.corpus.stopwords.words('english')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [3]:
print(stopword)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [6]:
embeddings_dict = {}
with open("/content/drive/MyDrive/PRISM/glove.6B.300d.txt", 'r', encoding="utf-8") as f:
  for line in f:
    values = line.split()
    word = values[0]
    vector = np.asarray(values[1:], "float32")
    embeddings_dict[word] = vector    

In [7]:
def find_closest_embeddings(embedding):
    return sorted(embeddings_dict.keys(), key=lambda word: spatial.distance.euclidean(embeddings_dict[word], embedding))

In [9]:
def get_GLOVE_augmented_text(text):
  x=len(text)
  tfidf=TfidfVectorizer()
  vc=tfidf.fit_transform(text).toarray()
  #print(vc)
  #print(tfidf.vocabulary_)
  words_lib=tfidf.vocabulary_
  new_corpus=[]
  for i in range(x):
    print(i)
    words=text[i]
    words=words.replace(".","")
    #print(words)  
    words=words.split(" ")
    new_words=""
    for m in words:
      if m.lower() not in stopword:
        if words_lib.__contains__(m):
          if vc[i][words_lib[m.lower()]]<0.10:
            m=m.lower()
            if embeddings_dict.__contains__(m):
              replaced_word=find_closest_embeddings(embeddings_dict[m.lower()])[1]
              new_words+=(replaced_word)
            else:
              new_words+=(m.lower())
          else:
            new_words+=(m.lower())
        else:
          new_words+=m.lower()
      else:
        new_words+=(m.lower())
      new_words+=" "
    #new_words=(str(new_words))
    new_corpus.append(new_words)
    new_corpus.append(text[i])
  return new_corpus

In [10]:
df=pd.read_csv('/content/drive/MyDrive/PRISM/IMDB Dataset.csv')

In [11]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [12]:
X=df['review']
y=df['sentiment']

In [13]:
X=X.values.tolist()

In [14]:
X=X[:50]
y=y[:50]

In [15]:
X[0]

"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the fa

In [16]:
Augmented_text=get_GLOVE_augmented_text(X)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49


In [17]:
en_stopwords=set(stopwords.words('english'))
wn = nltk.WordNetLemmatizer()

In [18]:
def getCleanedReview(review):
    review=review.lower()
    text_lc = "".join([word.lower() for word in review if word not in string.punctuation])
    #print(text_lc)
    text_rc=re.sub('[0-9]+',' ',text_lc)
    #tokenize
    tokens=word_tokenize(text_rc,language='english')
    #print(tokens)
    new_tokens=[token for token in tokens if token not in en_stopwords]
    #print(new_tokens)
    lemmatized_tokens=[wn.lemmatize(token) for token in new_tokens]
    return lemmatized_tokens

In [19]:
original=[]
y_augmented=[]
X_train=[]
X_augmented_train=[]

In [21]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [22]:
for i in range(len(X)):
    X_train.append(getCleanedReview(X[i]))
    if y[i] == 'positive':
        original.append(1)
        y_augmented.append(1)
        y_augmented.append(1)
    else:
        original.append(0)
        y_augmented.append(0)
        y_augmented.append(0)


In [23]:
for i in range(len(Augmented_text)):
    X_augmented_train.append(getCleanedReview(Augmented_text[i]))

In [24]:
df_new=[]
df_new=pd.DataFrame(df_new)

In [25]:
df_new["cleaned_review"]=X_train
X_train=df_new["cleaned_review"]


In [26]:
df_new_=[]
df_new_=pd.DataFrame(df_new_)
df_new_["cleaned_augmented_review"]=X_augmented_train
X_augmented_train=df_new_["cleaned_augmented_review"]

In [27]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
import sklearn.metrics

In [45]:
X_train.dropna()
cvt = CountVectorizer(analyzer=getCleanedReview,max_features=2000)
X_v = cvt.fit_transform(X_train.apply(lambda x: np.str_(x)))
X_augmented_v=cvt.fit_transform(X_augmented_train.apply(lambda x: np.str_(x)))

In [46]:
y_train=np.array(original)
y_augmented=np.array(y_augmented)

In [47]:
gnb=GaussianNB()
X_v=X_v.toarray()
X_augmented_v=X_augmented_v.toarray()
gnb.fit(X_v,y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [48]:
gnb2=GaussianNB()
gnb2.fit(X_augmented_v,y_augmented)

GaussianNB(priors=None, var_smoothing=1e-09)

In [73]:
X_test=df['review'].iloc[26000:26050]
y_test=df['sentiment'].iloc[26000:26050]

In [74]:
X_test=X_test.values.tolist()

In [75]:
X_test_new=[]
y_test_new=[]

In [76]:
for i in range(len(X_test)):
    X_test_new.append(getCleanedReview(X_test[i]))
    if y_test[26000+i] == 'positive':
        y_test_new.append(1)
    else:
        y_test_new.append(0)

In [77]:
df_new["cleaned_test_review"]=X_test_new
X_test=df_new["cleaned_test_review"]

In [78]:
X_test.dropna()
cvt1 = CountVectorizer(analyzer=getCleanedReview,max_features=2000)
X_test_v = cvt1.fit_transform(X_test.apply(lambda x: np.str_(x)))

In [79]:
y_test=np.array(y_test_new)

In [80]:
X_test_v=X_test_v.toarray()

In [81]:
gnb.score(X_test_v,y_test)

0.46

In [82]:
gnb2.score(X_test_v,y_test)

0.56